In [26]:
from nltk.corpus.reader.tagged import TaggedCorpusView
from sklearn.utils.extmath import randomized_svd
from collections import Counter
import numpy as np
import nltk
nltk.download('brown')
from nltk.corpus import brown

nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\chatu\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chatu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
documents = [brown.words(fileid) for fileid in brown.fileids()]
stopwords_list = stopwords.words('english')

In [28]:
def create_tfidf_matrix(documents: list[TaggedCorpusView], stopwords: list[str]) -> tuple[np.ndarray, list[str]]:
    # Preprocessing documents
    processed_docs = [
        [word.lower() for word in doc if word.isalnum() and word.lower() not in stopwords] 
        for doc in documents
    ]
    # Create sorted vocabulary and index mapping
    vocabulary = sorted({word for doc in processed_docs for word in doc})
    vocab_index = {word: i for i, word in enumerate(vocabulary)}
    num_docs, num_words = len(documents), len(vocabulary)
    
    # Initialize term_frequency_matrix and document frequencies
    tf_matrix = np.zeros((num_docs, num_words))
    doc_freq = np.zeros(num_words)
    
    for i, doc in enumerate(processed_docs):
        word_counts = Counter(doc)
        for word, count in word_counts.items():
            j = vocab_index[word]
            tf_matrix[i, j] = count
            doc_freq[j] += 1
    
    # Compute smoothened IDF using formula log10(N / (df + 1)) + 1
    idf = np.log10(num_docs / (doc_freq + 1)) + 1
    tfidf_matrix = tf_matrix * idf
    
    return tfidf_matrix, vocabulary


def get_idf_values(documents : list[TaggedCorpusView], stopwords: list[str]) -> np.ndarray:
    processed_docs = [
        [word.lower() for word in doc if word.isalnum() and word.lower() not in stopwords] 
        for doc in documents
    ]
    vocabulary = sorted({word for doc in processed_docs for word in doc})
    vocab_index = {word: i for i, word in enumerate(vocabulary)}
    doc_freq = np.zeros(len(vocabulary))
    
    for doc in processed_docs:
        word_counts = Counter(doc)
        for word, _ in word_counts.items():
            j = vocab_index[word]
            doc_freq[j] += 1
    
    idf = np.log10(len(documents) / (doc_freq + 1)) + 1
    return idf


def calculate_sparsity(tfidf_matrix: np.ndarray) -> float:
    sparsity = (tfidf_matrix == 0).sum() / tfidf_matrix.size
    return sparsity


def extract_salient_words(VT: np.ndarray, vocabulary: list[str], k: int) -> dict[int, list[str]]:
    n = VT.shape[0]
    salient_words = {}
    
    for dim in range(n):        
        # Sort the weights of each latent dimension
        sorted_indices = np.argsort(VT[dim, :])
        top_k_indices = sorted_indices[-k:]
        
        # Convert indices to words
        salient_words[dim] = [vocabulary[idx] for idx in top_k_indices]
    
    return salient_words

In [29]:
def cos_sim(A: np.ndarray, B: np.ndarray) -> float:
    dot_product = np.dot(A, B)
    norm1 = np.linalg.norm(A)
    norm2 = np.linalg.norm(B)
    cosine_similarity = 0.0 if norm1 == 0 or norm2 == 0 else dot_product / (norm1 * norm2)

    return cosine_similarity


def get_similar_documents(U: np.ndarray, Sigma: np.ndarray, VT: np.ndarray, doc_index: int, k: int) -> list[int]:
    doc_embeddings = U * Sigma
    query_embedding = doc_embeddings[doc_index]
    
    # Calculate cosine similarity between query document and all documents
    similarities = []
    for i in range(len(doc_embeddings)):
        if i != doc_index:  # Exclude the query document
            sim = cos_sim(query_embedding, doc_embeddings[i])
            similarities.append((i, sim))
    
    # Sort by similarity in descending order and get top k
    similarities.sort(key=lambda x: x[1], reverse=True)
    similar_doc_indices = [idx for idx, _ in similarities[:k]]
    
    return similar_doc_indices

In [ ]:
def document_retrieval(vocabulary: list[str], idf_values: np.ndarray, U: np.ndarray, Sigma: np.ndarray, VT: np.ndarray, query: list[str], k: int) -> list[int]:
    # Create vocabulary index mapping and process query
    vocab_index = {word: i for i, word in enumerate(vocabulary)}
    query_counts = Counter(w.lower() for w in query if w.isalnum())
    
    # Construct query TF-IDF vector (vocabulary size)
    q_tfidf = np.zeros(len(vocabulary))
    for word, count in query_counts.items():
        if word in vocab_index:
            q_tfidf[vocab_index[word]] = count * idf_values[vocab_index[word]]
    
    # Project query into 10-dimensional LSA space
    # q_tfidf (1 × vocab_size) @ VT.T (vocab_size × 10) = (1 × 10)
    q_lsa = q_tfidf @ VT.T
    
    # Scale query by singular values
    q_embedding = q_lsa / Sigma
    
    # Get document vectors in LSA space
    # U (n_docs × 10) already represents documents in LSA space
    doc_embeddings = U * Sigma  # Broadcasting Sigma across columns
    
    # Compute cosine similarities
    similarities = np.array([cos_sim(q_embedding, doc) for doc in doc_embeddings])
    
    # Return indices of top k most similar documents
    return similarities.argsort()[-k:].tolist()

In [31]:
# This will take a few minutes to run
tfidf_matrix, vocabulary = create_tfidf_matrix(documents, stopwords_list)
idf_values = get_idf_values(documents, stopwords_list)

print(tfidf_matrix.shape)
# (500, 40881)
print(tfidf_matrix[np.nonzero(tfidf_matrix)][:5])
# [5.96857651 2.1079054  3.         2.07572071 2.69897   ]
print(vocabulary[2000:2010])
# ['amoral', 'amorality', 'amorist', 'amorous', 'amorphous', 'amorphously', 'amortization', 'amortize', 'amory', 'amos']
print(calculate_sparsity(tfidf_matrix))
# 0.9845266994447298

(500, 40881)
[5.96857651 2.1079054  3.         2.07572071 2.69897   ]
['amoral', 'amorality', 'amorist', 'amorous', 'amorphous', 'amorphously', 'amortization', 'amortize', 'amory', 'amos']
0.9845266994447298


In [32]:
U, Sigma, VT = randomized_svd(tfidf_matrix, n_components=10, n_iter=100, random_state=42)
salient_words = extract_salient_words(VT, vocabulary, 10)
print(salient_words[1])
# ['anode', 'space', 'theorem', 'v', 'q', 'c', 'p', 'operator', 'polynomial', 'af']

['anode', 'space', 'theorem', 'v', 'q', 'c', 'p', 'operator', 'polynomial', 'af']


In [38]:
print("We will fetch documents similar to document {} - {}...".format(3, ' '.join(documents[3][:50])))
# We will fetch documents similar to document 3 - 
# Oslo The most positive element to emerge from the Oslo meeting of North Atlantic Treaty Organization Foreign Ministers has been the freer , 
# franker , and wider discussions , animated by much better mutual understanding than in past meetings . This has been a working session of an organization that...

similar_doc_indices = get_similar_documents(U, Sigma, VT, 3, 5)
for i in range(2):
    print("Document {} is similar to document 3 - {}...".format(similar_doc_indices[i], ' '.join(documents[similar_doc_indices[i]][:50])))
# Document 61 is similar to document 3 - 
# For a neutral Germany Soviets said to fear resurgence of German militarism to the editor of the New York Times : 
# For the first time in history the entire world is dominated by two large , powerful nations armed with murderous nuclear weapons that make conventional warfare of the past...
# Document 6 is similar to document 3 - 
# Resentment welled up yesterday among Democratic district leaders and some county leaders at reports that Mayor Wagner had decided to seek a third term with Paul R. Screvane and Abraham D. Beame as running mates . 
# At the same time reaction among anti-organization Democratic leaders and in the Liberal party... 

We will fetch documents similar to document 3 - Oslo The most positive element to emerge from the Oslo meeting of North Atlantic Treaty Organization Foreign Ministers has been the freer , franker , and wider discussions , animated by much better mutual understanding than in past meetings . This has been a working session of an organization that...
Document 61 is similar to document 3 - For a neutral Germany Soviets said to fear resurgence of German militarism to the editor of the New York Times : For the first time in history the entire world is dominated by two large , powerful nations armed with murderous nuclear weapons that make conventional warfare of the past...
Document 6 is similar to document 3 - Resentment welled up yesterday among Democratic district leaders and some county leaders at reports that Mayor Wagner had decided to seek a third term with Paul R. Screvane and Abraham D. Beame as running mates . At the same time reaction among anti-organization Democratic leaders and

In [55]:
query = ['Krim', 'attended', 'the', 'University', 'of', 'North', 'Carolina', 'to', 'follow', 'Thomas', 'Wolfe']
print("We will fetch documents relevant to query - {}".format(' '.join(query)))
relevant_doc_indices = document_retrieval(vocabulary, idf_values, U, Sigma, VT, query, 5)
for i in range(2):
    print("Document {} is relevant to query - {}...".format(relevant_doc_indices[i], ' '.join(documents[relevant_doc_indices[i]][:50])))
# Document 90 is relevant to query - 
# One hundred years ago there existed in England the Association for the Promotion of the Unity of Christendom . 
# Representing as it did the efforts of only unauthorized individuals of the Roman and Anglican Churches , and urging a communion of prayer unacceptable to Rome , this association produced little...
# Document 101 is relevant to query - To what extent and in what ways did Christianity affect the United States of America in the nineteenth century ? ? 
# How far and in what fashion did it modify the new nation which was emerging in the midst of the forces shaping the revolutionary age ? ? To what...


We will fetch documents relevant to query - Krim attended the University of North Carolina to follow Thomas Wolfe
Document 90 is relevant to query - One hundred years ago there existed in England the Association for the Promotion of the Unity of Christendom . Representing as it did the efforts of only unauthorized individuals of the Roman and Anglican Churches , and urging a communion of prayer unacceptable to Rome , this association produced little...
Document 96 is relevant to query - Few persons who join the Church are insincere . They earnestly desire to do the will of God . When they fall by the wayside and fail to achieve Christian stature , it is an indictment of the Church . These fatalities are dramatic evidence of `` halfway evangelism ''...
